In [ ]:
import os
import unicodedata
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
def is_english_only(string):
    for s in string:
        cat = unicodedata.category(s)         
        if not cat in ['Ll', 'Lu', 'Nd', 'Po', 'Pd', 'Zs']:
            return False
    return True

In [ ]:
df = pd.read_parquet('/kaggle/input/diffusiondb-metadata/metadata.parquet', columns=['image_name', 'prompt', 'width', 'height'])
df = df[(df['width'] == 512) & (df['height'] == 512)]
df['prompt'] = df['prompt'].str.strip()
df = df[df['prompt'].map(lambda x: len(x.split())) >= 5]
df = df[~df['prompt'].str.contains('^(?:\s*|NULL|null|NaN)$', na=True)]
df = df[df['prompt'].apply(is_english_only)]
df['head'] = df['prompt'].str[:15]
df['tail'] = df['prompt'].str[-15:]
df.drop_duplicates(subset='head', inplace=True)
df.drop_duplicates(subset='tail', inplace=True)
df.reset_index(drop=True, inplace=True)

for i in tqdm(range(1, 2000, 100)):
    image_dir = f'/kaggle/input/diffusiondb-2m-part-{str(i).zfill(4)}-to-{str(i+99).zfill(4)}-of-2000/'
    images = os.listdir(image_dir)
    df.loc[df['image_name'].isin(images), 'filepath'] = image_dir + df['image_name']

df = df[['filepath', 'prompt']].copy()
assert not df['filepath'].isnull().any()
df

In [ ]:
df.to_csv('diffusiondb.csv', index=False)